Gavin Gunawardena \
INFS 770 \
Assignment 3 \
Dr. Liu


In [14]:
#Library Imports
import pandas as pd
import numpy as np
import gensim
import os
import re
from gensim.models import LdaModel, LsiModel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Task 1: Import Data
docs = []
directory = os.getcwd() + "\\tp_dataset"
for filename in os.listdir(directory):
    f = open(os.path.join(directory, filename), "r")
    docs.append(f.read())

#print number of items in the list to confirm whether it matches up
#with the number of text files
print(len(docs))


9


In [3]:
#Task 2: Write code to tokenize the docs using the TF-IDF vectorizer

def before_token(documents):
    # conver words to lower case
    lower = map(str.lower, documents)
    # remove puntuations
    punctuationless = list(map(lambda x: " ".join(re.findall('\\b\\w\\w+\\b',x)), lower))
    # remove numbers
    return list(map(lambda x:re.sub('\\b[0-9]+\\b', '', x), punctuationless))
docs1 = before_token(docs)
import nltk
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t,"v") for t in word_tokenize(doc)]
stopwords = nltk.corpus.stopwords.words("english")
#remove words that appear in only one or two documents
#remove words that appears in over 50% of documents
vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),norm='l2',stop_words=stopwords, min_df=2, max_df=0.5)
#vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),stop_words=stopwords)
corpus_vect = vectorizer.fit_transform(docs1)
df_vect = pd.DataFrame(corpus_vect.toarray(), columns=vectorizer.get_feature_names())
#Print features
print(df_vect)


       able    accord       act    action  actually  additional    adjust  \
0  0.000000  0.025333  0.032976  0.000000  0.025333    0.032976  0.000000   
1  0.000000  0.080357  0.000000  0.000000  0.026786    0.000000  0.000000   
2  0.054252  0.000000  0.000000  0.054252  0.041677    0.000000  0.000000   
3  0.000000  0.056784  0.000000  0.000000  0.000000    0.000000  0.000000   
4  0.000000  0.000000  0.033851  0.000000  0.000000    0.033851  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000    0.000000  0.000000   
6  0.062734  0.000000  0.000000  0.000000  0.048194    0.000000  0.000000   
7  0.000000  0.066133  0.000000  0.043043  0.000000    0.000000  0.086086   
8  0.000000  0.000000  0.000000  0.000000  0.000000    0.000000  0.098844   

   advisers    africa       age  ...    wealth   wealthy     weeks     white  \
0  0.000000  0.000000  0.000000  ...  0.028671  0.032976  0.025333  0.126664   
1  0.121264  0.000000  0.000000  ...  0.030316  0.069734  0.080357  0

C:\Users\ggobl\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'win', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Task 3: Create a textbox and use your own language to discuss what TF-IDF means.

*The idea of Term Frequency – Inverse Document Frequency is that if specific words occur in just a few documents, they might be useful for distinguishing some documents from others. This model utilizes the inverse document frequency formula to assign weights to words based on the total number of documents and how many documents contain the word.*

In [4]:
#Task 4: Write code to convert the vectorized data to a gensim corpus object. 
#Print id2word
from gensim import corpora
word2id = dict((k, v) for k, v in vectorizer.vocabulary_.items())
id2word = dict((v, k) for k,v in vectorizer.vocabulary_.items())
d=corpora.Dictionary()
d.id2token = id2word
d.token2id = word2id
corpus = gensim.matutils.Sparse2Corpus(corpus_vect, documents_columns=False)
print(id2word)


{412: 'white', 166: 'house', 241: 'officials', 268: 'prepare', 269: 'present', 270: 'president', 36: 'biden', 391: 'trillion', 176: 'infrastructure', 185: 'job', 247: 'package', 280: 'profile', 106: 'domestic', 262: 'policy', 275: 'priorities', 148: 'free', 66: 'community', 64: 'college', 398: 'universal', 266: 'prekindergarten', 1: 'accord', 380: 'three', 132: 'familiar', 179: 'internal', 82: 'coronavirus', 311: 'relief', 257: 'piece', 238: 'next', 209: 'major', 203: 'legislative', 139: 'final', 288: 'push', 51: 'build', 31: 'back', 34: 'better', 11: 'agenda', 118: 'effort', 47: 'break', 143: 'focus', 56: 'child', 372: 'tax', 335: 'several', 417: 'years', 347: 'speak', 72: 'condition', 96: 'describe', 276: 'private', 81: 'conversations', 360: 'stress', 364: 'subject', 55: 'change', 12: 'aid', 396: 'unclear', 358: 'still', 144: 'follow', 411: 'weeks', 325: 'second', 74: 'confusion', 19: 'among', 76: 'congressional', 385: 'top', 37: 'bill', 131: 'face', 95: 'democrats', 322: 'scale', 31

In [5]:
#Task 5: Write code to compute coherence scores for different numbers of topics including 2, 3, 4, and 5. 
from gensim.models import CoherenceModel
for i in [1,2,3,4,5,6,7]:
    cs = 0
    for j in range(30):
        lda = LdaModel(corpus, num_topics=i,id2word=id2word, passes=50)
        #print(lda.print_topics())
        coherence_model_lda = CoherenceModel(model=lda, corpus=corpus, dictionary=d, coherence='u_mass')
        coherence_lda = coherence_model_lda.get_coherence()
        cs += coherence_lda
    print('Coherence Score for %d topics: %f' % (i,cs/30))

Coherence Score for 1 topics: -17.093861
Coherence Score for 2 topics: -7.251259
Coherence Score for 3 topics: -4.578328
Coherence Score for 4 topics: -3.783313
Coherence Score for 5 topics: -3.619911
Coherence Score for 6 topics: -3.219937
Coherence Score for 7 topics: -3.599352


In [6]:
#Task 6: You usually cannot rely on the coherence scores to determine the optional 
#number of topics. Next, you want to further compare the top 3 numbers 
#of topics with the highest coherence scores.
for i in [4,6,7]:
    lda = LdaModel(corpus, num_topics=i,id2word=id2word, random_state=10, passes=50) #
    lda_docs = lda[corpus]
    scores = np.round([[doc[1] for doc in row] for row in lda_docs], 3)
    topics = list(range(1,i+1,1))
    col_headers = map(lambda x: 'topic: ' + str(x), topics)
    df_lda = pd.DataFrame(scores, columns=list(col_headers))
    print('%d topics:' % (i))
    print(lda.print_topics())
    print('Matrix for %d topics:' % (i))
    display(df_lda)

    

4 topics:
[(0, '0.008*"apple" + 0.008*"sensor" + 0.008*"temperature" + 0.007*"home" + 0.006*"smart" + 0.005*"homepod" + 0.005*"humidity" + 0.005*"amazon" + 0.005*"room" + 0.004*"device"'), (1, '0.002*"device" + 0.002*"launch" + 0.002*"speakers" + 0.002*"mini" + 0.002*"feature" + 0.002*"software" + 0.002*"speaker" + 0.002*"bloomberg" + 0.002*"tv" + 0.002*"rather"'), (2, '0.002*"device" + 0.002*"launch" + 0.002*"speakers" + 0.002*"mini" + 0.002*"software" + 0.002*"feature" + 0.002*"speaker" + 0.002*"bloomberg" + 0.002*"rather" + 0.002*"tv"'), (3, '0.011*"vaccine" + 0.008*"biden" + 0.008*"infrastructure" + 0.006*"astrazeneca" + 0.006*"trial" + 0.005*"tax" + 0.005*"data" + 0.005*"house" + 0.005*"homepod" + 0.005*"spend"')]
Matrix for 4 topics:


,topic: 1,topic: 2,topic: 3,topic: 4
0,0.023,0.023,0.023,0.931
1,0.025,0.025,0.025,0.925
2,0.025,0.025,0.025,0.926
3,0.028,0.028,0.028,0.915
4,0.026,0.026,0.026,0.923
5,0.032,0.032,0.032,0.904
6,0.047,0.045,0.045,0.864
7,0.902,0.032,0.032,0.033
8,0.890,0.037,0.037,0.037


6 topics:
[(0, '0.009*"vaccine" + 0.007*"sensor" + 0.006*"study" + 0.005*"smart" + 0.005*"home" + 0.005*"temperature" + 0.005*"astrazeneca" + 0.005*"apple" + 0.004*"countries" + 0.004*"case"'), (1, '0.008*"apple" + 0.007*"temperature" + 0.006*"home" + 0.006*"sensor" + 0.006*"homepod" + 0.005*"device" + 0.004*"thermostat" + 0.004*"feature" + 0.004*"software" + 0.004*"mini"'), (2, '0.009*"vaccine" + 0.006*"trial" + 0.005*"astrazeneca" + 0.005*"oxford" + 0.005*"unite" + 0.004*"age" + 0.004*"efficacy" + 0.004*"data" + 0.004*"medicine" + 0.004*"older"'), (3, '0.012*"biden" + 0.011*"infrastructure" + 0.008*"tax" + 0.007*"house" + 0.007*"homepod" + 0.007*"spend" + 0.006*"trillion" + 0.006*"white" + 0.005*"code" + 0.005*"proposal"'), (4, '0.008*"vaccine" + 0.007*"data" + 0.007*"trial" + 0.006*"dose" + 0.006*"efficacy" + 0.005*"give" + 0.004*"weeks" + 0.004*"longer" + 0.004*"interval" + 0.004*"fda"'), (5, '0.002*"suggest" + 0.002*"find" + 0.002*"launch" + 0.002*"speakers" + 0.002*"mean" + 0.002

,topic: 1,topic: 2,topic: 3,topic: 4,topic: 5,topic: 6
0,0.015,0.015,0.015,0.924,0.015,0.015
1,0.016,0.016,0.016,0.919,0.016,0.016
2,0.016,0.016,0.016,0.919,0.016,0.016
3,0.019,0.019,0.907,0.019,0.019,0.019
4,0.915,0.017,0.017,0.017,0.017,0.017
5,0.021,0.021,0.021,0.021,0.895,0.021
6,0.029,0.029,0.029,0.853,0.029,0.029
7,0.022,0.893,0.021,0.021,0.021,0.021
8,0.878,0.024,0.024,0.024,0.024,0.024


7 topics:
[(0, '0.009*"vaccine" + 0.008*"apple" + 0.006*"temperature" + 0.006*"study" + 0.006*"home" + 0.006*"sensor" + 0.006*"astrazeneca" + 0.005*"homepod" + 0.005*"countries" + 0.004*"device"'), (1, '0.009*"vaccine" + 0.008*"data" + 0.007*"trial" + 0.007*"dose" + 0.007*"efficacy" + 0.005*"give" + 0.004*"weeks" + 0.004*"longer" + 0.004*"interval" + 0.004*"fda"'), (2, '0.010*"vaccine" + 0.006*"trial" + 0.006*"astrazeneca" + 0.005*"oxford" + 0.005*"unite" + 0.004*"age" + 0.004*"efficacy" + 0.004*"data" + 0.004*"medicine" + 0.004*"older"'), (3, '0.014*"biden" + 0.013*"infrastructure" + 0.009*"tax" + 0.008*"house" + 0.008*"spend" + 0.007*"trillion" + 0.006*"white" + 0.006*"proposal" + 0.006*"next" + 0.005*"republicans"'), (4, '0.008*"sensor" + 0.007*"smart" + 0.007*"home" + 0.007*"temperature" + 0.006*"apple" + 0.005*"currently" + 0.005*"trail" + 0.005*"amazon" + 0.005*"room" + 0.005*"humidity"'), (5, '0.012*"homepod" + 0.007*"software" + 0.007*"feature" + 0.007*"code" + 0.006*"apple" + 

,topic: 1,topic: 2,topic: 3,topic: 4,topic: 5,topic: 6,topic: 7
0,0.013,0.013,0.013,0.922,0.013,0.013,0.013
1,0.014,0.014,0.014,0.916,0.014,0.014,0.014
2,0.014,0.014,0.014,0.917,0.014,0.014,0.014
3,0.016,0.016,0.905,0.016,0.016,0.016,0.016
4,0.913,0.015,0.015,0.015,0.014,0.014,0.014
5,0.018,0.892,0.018,0.018,0.018,0.018,0.018
6,0.025,0.025,0.025,0.025,0.025,0.850,0.025
7,0.890,0.018,0.018,0.018,0.018,0.018,0.018
8,0.021,0.021,0.021,0.021,0.875,0.021,0.021


Task 7: Create a textbox and tell me among the top 3 numbers with the highest average coherence scores, which one gives you the best topic modeling results. 

*7 topics, as it gives more leeway for the chance that each or most of the documents are based on a different or slightly different topic, or consist of different subtopics. This hypothesis is reinforced by the fact that the document score table presents a score of .8 or more for each of the topic numbers except for 7. Also this detected pattern in the 7 topics column could indicate a 7th undetected topic that is a combination of subtopics and that would've been missed with a lower topic count.*

In [10]:
#Task 8: 	Run truncated SVD(set n_components to be the optimal number 
#of topics you found in T7) and print topics and the topic/document table.
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=7)
tsvd.fit(corpus_vect)
df_comp = pd.DataFrame(tsvd.components_, columns=vectorizer.get_feature_names())
df_comp = df_comp.apply(lambda x: np.round(x,3))
#print topics
print('topics')
display(df_comp)
#print topic/document table
scores = np.round(np.round(tsvd.transform(corpus_vect), 3))
df_tsvd = pd.DataFrame(scores, columns=["topic 1", "topic 2","topic 3", "topic 4","topic 5", "topic 6", "topic 7"])
print('topic/document table:')
display(df_tsvd)

topics


,able,accord,act,action,actually,additional,adjust,advisers,africa,age,...,wealth,wealthy,weeks,white,wing,world,worry,year,years,york
0,0.020,0.049,0.017,0.020,0.031,0.017,0.016,0.069,0.025,0.062,...,0.031,0.030,0.065,0.127,0.040,0.043,0.011,0.026,0.053,0.026
1,-0.012,-0.003,0.003,-0.012,-0.020,0.003,0.001,-0.038,0.039,0.098,...,-0.024,-0.022,0.031,-0.082,-0.030,0.066,0.011,-0.017,0.021,-0.020
2,0.014,0.020,-0.004,0.017,0.007,-0.004,0.083,-0.017,-0.007,-0.018,...,-0.008,-0.007,-0.017,-0.032,-0.010,-0.012,0.019,0.012,-0.015,-0.006
3,0.057,-0.011,0.001,-0.005,0.043,0.001,-0.061,-0.005,-0.003,-0.007,...,-0.001,-0.002,-0.004,-0.007,-0.002,-0.002,-0.004,-0.004,-0.003,-0.002
4,-0.003,0.009,-0.036,-0.007,0.002,-0.036,0.001,-0.017,-0.004,-0.040,...,-0.000,0.011,0.143,0.010,0.005,-0.094,-0.036,-0.007,0.069,0.001
5,-0.054,0.081,0.002,-0.057,-0.014,0.002,-0.003,0.063,-0.007,-0.014,...,-0.016,0.080,0.090,0.040,0.021,0.016,0.007,-0.056,-0.071,-0.012
6,-0.002,0.112,-0.034,0.002,0.002,-0.034,-0.000,0.006,0.051,0.125,...,0.001,0.014,-0.115,0.059,0.010,-0.022,-0.023,-0.004,0.017,0.007


topic/document table:


,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7
0,1.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0
1,1.0,-0.0,-0.0,-0.0,0.0,0.0,0.0
2,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,1.0,1.0,-0.0,-0.0,-0.0,-0.0,0.0
4,1.0,1.0,-0.0,0.0,-0.0,0.0,-0.0
5,0.0,1.0,-0.0,-0.0,0.0,-0.0,-0.0
6,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,0.0,0.0,1.0,-0.0,-0.0,-0.0,0.0
8,0.0,0.0,1.0,-0.0,0.0,0.0,-0.0


In [13]:
#Task 8b: Using the LsiModel class to implement truncated SVD
lsi = LsiModel(corpus=corpus, id2word=id2word, num_topics=7)


#print topics
display(lsi.print_topics(7))
lsi_docs = lsi[corpus]

#print topic/document table
scores = np.round([[doc[1] for doc in row] for row in lsi_docs], 3)
df_lsi = pd.DataFrame(scores, columns=["topic 1", "topic 2","topic 3", "topic 4","topic 5", "topic 6", "topic 7"])
print('topic/document table:')
display(df_lsi)

[(0,
  '0.326*"biden" + 0.318*"vaccine" + 0.292*"infrastructure" + 0.185*"tax" + 0.177*"house" + 0.173*"spend" + 0.135*"astrazeneca" + 0.129*"trillion" + 0.128*"trial" + 0.127*"white"'),
 (1,
  '0.502*"vaccine" + -0.233*"biden" + -0.221*"infrastructure" + 0.211*"astrazeneca" + 0.209*"trial" + 0.193*"data" + 0.176*"efficacy" + 0.153*"dose" + -0.139*"tax" + -0.133*"house"'),
 (2,
  '-0.403*"apple" + -0.335*"homepod" + -0.313*"sensor" + -0.291*"temperature" + -0.271*"home" + -0.188*"smart" + -0.173*"software" + -0.173*"feature" + -0.157*"device" + -0.146*"humidity"'),
 (3,
  '-0.486*"homepod" + -0.275*"software" + -0.275*"feature" + -0.267*"code" + 0.240*"sensor" + -0.222*"could" + 0.191*"temperature" + 0.187*"home" + -0.175*"run" + 0.171*"smart"'),
 (4,
  '-0.358*"trial" + -0.344*"data" + 0.244*"study" + -0.225*"efficacy" + -0.200*"dose" + 0.196*"astrazeneca" + -0.185*"give" + 0.155*"countries" + -0.143*"weeks" + 0.142*"johnson"'),
 (5,
  '0.242*"proposal" + 0.226*"move" + -0.205*"infras

topic/document table:


,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7
0,0.752,-0.492,0.146,-0.011,0.022,-0.076,-0.051
1,0.722,-0.472,0.134,0.029,-0.083,0.451,0.081
2,0.719,-0.500,0.141,0.013,0.044,-0.370,-0.016
3,0.507,0.742,0.129,0.036,0.023,-0.074,0.407
4,0.517,0.676,0.092,-0.007,0.454,0.097,-0.225
5,0.484,0.693,0.112,0.012,-0.476,-0.029,-0.211
6,0.188,0.022,-0.556,-0.799,-0.020,0.007,0.007
7,0.234,0.018,-0.916,0.080,0.014,-0.014,0.032
8,0.190,0.001,-0.823,0.463,-0.015,0.003,-0.029
